<a href="https://colab.research.google.com/github/ptse8204/airlinedatabias/blob/main/dsc180b_converters_func.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

This modules produced in this notebook should be able to grab all the neccessary module for a smooth navigation and use of the dataset along the road

# Survey -- ticket useable functions

The ticket.py was updated from ticket_eda

Naming convention -- not yet resolved

In [ ]:
#@title ticket.py_file
%%writefile ticket.py
#### this py file contains EDA functions for Ticket Datasets (2018 - 2022)

# Dependency
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#Joseph
def flight_length_bar(dataset):
  #Flight length
  def flight_length(x):
    if x <= 1725: #miles
        return "short-haul"
    elif x>1725 and x<=3450:
        return "medium-haul"
    else:
        return "long-haul"

  #adding new column to data and plotting bar chart
  dataset["flight_length"]=dataset["MilesFlown"].apply(lambda x: flight_length(x))
  lengths=dataset.groupby("flight_length").count()["ItinID"]
  plt.bar(x=lengths.index,height=lengths)
  plt.title("Flight Length Count")

#adjusting fare columns for inflation 
def inflation(dataset,inflation_amount):
    #inflation amount in decimals
    #ex: 2019 to 2022 = 14.5% ->0.145
    dataset["FarePerMile"]=dataset["FarePerMile"].apply(lambda x: x + (x*inflation_amount)) 
    dataset["ItinFare"]=dataset["ItinFare"].apply(lambda x: x + (x*inflation_amount)) 
    return dataset



### Qixi Huang

### $1 in 2018 is worth $1.16 in 2022
def adjust_inflation_2018(dataset):
    dataset['ItinFare'] = dataset['ItinFare'].apply(lambda x: x * 1.16)
    dataset['FarePerMile'] = dataset['FarePerMile'].apply(lambda x: x * 1.16)

def Market_share_Carrier(dataset):
    ### plot bars chart that show market share for each airlines
    carrier_percent = dataset.RPCarrier.value_counts()
    carrier_percent.plot.bar()

def Rev_Carrier(dataset):
    ### plot bars chart that show revenue per miles on each airlines
    avg_rev = dataset.groupby('RPCarrier')['FarePerMile'].mean()
    avg_rev.sort_values().plot.bar()

def fare_to_dis(dataset):
    ### bar charts that show fare per mile with respect to flight_length
    avg_rev_dis = dataset.groupby('flight_length')['FarePerMile'].mean()
    avg_rev_dis.plot.bar()

def carrier_option_dis(dataset):
    
    ### returns two dictionaries that shows whats the most and least profitable option
    ### in terms of flight length category, e.g. short hual maybe more profitable for some carriers.
    carrier_dis_table = dataset.groupby(['RPCarrier','flight_length'])['FarePerMile'].mean()
    
    carrier_percent = dataset.RPCarrier.value_counts()
    
    profit_dis_airline = dict()
    not_profit_dis_airline = dict()
    
    for i in carrier_percent.index:
        
        profit_dis_airline[i] = carrier_dis_table[i].idxmax()
        not_profit_dis_airline[i] = carrier_dis_table[i].idxmin()
        
    return [profit_dis_airline, not_profit_dis_airline]


# Garrick Su

import cpi 
from datetime import date

def inflation(row):
    month = 1 if row["Quarter"] == 1 else (3 if row["Quarter"] == 2 else (6 if row["Quarter"] == 3 else 9))
    row["ItinFare"] = cpi.inflate(row["ItinFare"], date(row["Year"], month, 1), to=date(2022, 1, 1))
    row["FarePerMile"] = cpi.inflate(row["FarePerMile"], date(row["Year"], month, 1), to=date(2022, 1, 1))
    return row

def convert_inflation(dataset):
    return dataset.apply(inflation, axis=1)

def clean_and_merge_race_city(L_CITY_MARKET_ID, race, dataset):
    race = race.T
    race.columns = race.iloc[0]
    race = race.drop(race.index[0]).reset_index()
    race["Metro Area"] = race['index'].apply(lambda x: x[-10:] == "Metro Area")
    race["Area Name"] = race['index'].apply(lambda x: x[:-11])
    race = race.merge(L_CITY_MARKET_ID, left_on="Area Name", right_on="Description", how='inner')
    return race, race.merge(dataset, left_on="Code", right_on="OriginCityMarketID", how='inner')

def lowest_and_highest_5(merged_dataset, merged_race):
    lowest_5 = merged_dataset.groupby("Code").mean()["ItinFare"].sort_values().iloc[0:5].index
    highest_5 = merged_dataset.groupby("Code").mean()["ItinFare"].sort_values().iloc[-5:].index
    return race[race["Code"].isin(lowest_5)], race[race["Code"].isin(highest_5)]





### Edwin
## Ticket, works with combined

# Drop fpm outliers
def filter_ticket_df_outliers(ticket_df,combined):
	return ticket_df[combined["FarePerMile"] < ticket_df["FarePerMile"].quantile(.99)]

# This function return average FarePerMile per Carrier
def avg_fpm(ticket_df):
    ticket_df.groupby("RPCarrier").mean()["FarePerMile"].plot(kind="bar")

# This function return average FarePerMile per Carrier for Legacy Airlines
def avg_fpm_legacy(ticket_df):
    ticket_df.groupby("RPCarrier").mean()["FarePerMile"].loc[[\
    "AA", "AS", "B6", "DL", "HA", "UA", "WN"]].plot(kind="barh")

Writing ticket.py


# Survey -- coupon useable functions

In [ ]:
#@title coupon.py file
%%writefile coupon.py
# Dependency
import pandas as pd
import numpy as np

### Edwin
## Coupon, works with combined
# This function allows filtering different classes
# accept input of a single class
def select_class(coupon_df, class_str):
  return coupon_df[coupon_df['FareClass'] == class_str]

# This function allows filtering only the class (X, Y)
def econ_class(coupon_df):
  return coupon_df[(coupon_df.FareClass == "X")|(coupon_df.FareClass == "Y")]


Writing coupon.py


# Survey -- combined functions

In [ ]:
#@title other.py file
%%writefile other.py

# Dependency
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### Edwin
## Combined
# This function would provide a table that grab columns
# that we need in both sets and return a useful df

# Essential this table would combine the 2 tables with only useful columns

def gen_ticket_coupon(ticket_df, coupon_df):
  ticket_df_reduced = ticket_df[["ItinID", "Coupons", 'Year', 'Quarter', 
                                 'Origin', 'OriginCityMarketID', 'OriginState',
                                 'RoundTrip', 'OnLine', 'DollarCred', 'FarePerMile',
                                  'RPCarrier', 'Passengers', 'ItinFare', 'BulkFare'
                                  , 'MilesFlown', 'ItinGeoType']]
  del ticket_df
  coupon_df_reduced = coupon_df[['ItinID','SeqNum', 'Coupons', 'Year', 
                                 'Quarter', 'DestCityMarketID', 'Dest', 
                                 'DestState', 'CouponGeoType', 'FareClass']]
  del coupon_df
  max_gp = coupon_df_reduced[["SeqNum", "ItinID"]].groupby("ItinID").max().reset_index()
  coupon_df_filter = coupon_df_reduced.merge(max_gp, on=["ItinID",	"SeqNum"])
  return ticket_df_reduced.merge(coupon_df_filter, on=['ItinID', 'Year', 'Quarter'])


### Edwin

## Connecting census dataset
# Reading Census City Data
# Depreciated
def read_cen_data(path):
  census_city_code = pd.read_csv(path)
  census_city_code["median_income"] = census_city_code["11"].str.replace(",", "").astype("int")
  return census_city_code

# Just a histogram for median income
def us_city_median_income_plot(census_df):
  census_df.median_income.plot(kind="hist")

### Edwin
## Census EDA with Airline Data, Prereq: Combined, Census

# This shows the data statistics of city areas that has 
# median income in the bottom 25 percentile
def bottom_25_data(combined, census_df):
    bot_25_median = census_df.median_income.quantile(.25)
    bottom_25_origins = combined[["FarePerMile", "RPCarrier", "OriginCityMarketID", 
                                  "MilesFlown", "SeqNum"]].merge(
      census_df[census_df.median_income <= bot_25_median][["Code", "median_income"]], 
      left_on = "OriginCityMarketID", right_on="Code")
    bottom_25_dest = combined[["FarePerMile", "RPCarrier", "DestCityMarketID", 
                               "MilesFlown", "SeqNum"]].merge(
      census_df[census_df.median_income <= bot_25_median][["Code", "median_income"]], 
      left_on = "DestCityMarketID", right_on="Code")
    print("Flights originate city areas that has median income in the bottom 25 percentile")
    print("Mean of FarePerMile : ", bottom_25_origins.FarePerMile.mean())
    print("Mean of MilesFlown : ", bottom_25_origins.MilesFlown.mean())
    print("Mean of Average Segments:", bottom_25_origins.SeqNum.mean())
    print("FarePerMile by carrier:")
    bottom_25_origins[["FarePerMile", "RPCarrier"]].groupby("RPCarrier").mean()["FarePerMile"].plot(kind="bar")
    plt.show()
    print("FarePerMile Distribution:")
    bottom_25_origins.FarePerMile.hist()
    plt.show()
    print("Flights destination is city areas that has median income in the bottom 25 percentile")
    print("Mean of FarePerMile : ", bottom_25_dest.FarePerMile.mean())
    print("Mean of MilesFlown : ", bottom_25_dest.MilesFlown.mean())
    print("Mean of Average Segments:", bottom_25_dest.SeqNum.mean())
    print("FarePerMile by carrier:")
    bottom_25_dest[["FarePerMile", "RPCarrier"]].groupby("RPCarrier").mean()["FarePerMile"].plot(kind="bar")
    plt.show()
    print("FarePerMile Distribution:")
    bottom_25_dest.FarePerMile.hist()
    plt.show()

# This shows the data statistics of city areas that has 
# median income in the upper 25 percentile
def upper_25_data(combined, census_df):
    bot_25_median = census_df.median_income.quantile(.75)
    upper_25_origins = combined[["FarePerMile", "RPCarrier", "OriginCityMarketID", 
                                  "MilesFlown", "SeqNum"]].merge(
      census_df[census_df.median_income >= bot_25_median][["Code", "median_income"]], 
      left_on = "OriginCityMarketID", right_on="Code")
    upper_25_dest = combined[["FarePerMile", "RPCarrier", "DestCityMarketID", 
                               "MilesFlown", "SeqNum"]].merge(
      census_df[census_df.median_income >= bot_25_median][["Code", "median_income"]], 
      left_on = "DestCityMarketID", right_on="Code")
    print("Flights originate city areas that has median income in the upper 25 percentile")
    print("Mean of FarePerMile : ", upper_25_origins.FarePerMile.mean())
    print("Mean of MilesFlown : ", upper_25_origins.MilesFlown.mean())
    print("Mean of Average Segments:", upper_25_origins.SeqNum.mean())
    print("FarePerMile by carrier:")
    upper_25_origins[["FarePerMile", "RPCarrier"]].groupby("RPCarrier").mean()["FarePerMile"].plot(kind="bar")
    plt.show()
    print("FarePerMile Distribution:")
    upper_25_origins.FarePerMile.hist()
    plt.show()
    print("Flights destination is city areas that has median income in the upper 25 percentile")
    print("Mean of FarePerMile : ", upper_25_dest.FarePerMile.mean())
    print("Mean of MilesFlown : ", upper_25_dest.MilesFlown.mean())
    print("Mean of Average Segments:", upper_25_dest.SeqNum.mean())
    print("FarePerMile by carrier:")
    upper_25_dest[["FarePerMile", "RPCarrier"]].groupby("RPCarrier").mean()["FarePerMile"].plot(kind="bar")
    plt.show()
    print("FarePerMile Distribution:")
    upper_25_dest.FarePerMile.hist()
    plt.show()


# This shows the flight statistics of city areas that has 
# with upper and lower 25th percentile as orgin and dest
# and vice-versa
def lower_and_upper_data(combined, census_df):
    bot_25_median = census_df.median_income.quantile(.25)
    up_25_median = census_df.median_income.quantile(.75)
    origins = combined[["FarePerMile", "RPCarrier", "OriginCityMarketID", 
                                  "DestCityMarketID", "MilesFlown", "SeqNum"]].merge(
      census_df[census_df.median_income <= bot_25_median][["Code", "median_income"]], 
      left_on = "OriginCityMarketID", right_on="Code")
    origin_dest = origins.merge(
      census_df[census_df.median_income >= up_25_median][["Code", "median_income"]], 
      left_on = "DestCityMarketID", right_on="Code")
    print("Flights originate bottom 25 to upper 25")
    print("Mean of FarePerMile : ", origin_dest.FarePerMile.mean())
    print("Mean of MilesFlown : ", origin_dest.MilesFlown.mean())
    print("Mean of Average Segments:", origin_dest.SeqNum.mean())
    print("FarePerMile by carrier:")
    origin_dest[["FarePerMile", "RPCarrier"]].groupby("RPCarrier").mean()["FarePerMile"].plot(kind="bar")
    plt.show()
    print("FarePerMile Distribution:")
    origin_dest.FarePerMile.hist()
    plt.show()
    origins = combined[["FarePerMile", "RPCarrier", "OriginCityMarketID", 
                                  "DestCityMarketID", "MilesFlown", "SeqNum"]].merge(
      census_df[census_df.median_income >= up_25_median][["Code", "median_income"]], 
      left_on = "OriginCityMarketID", right_on="Code")
    origin_dest = origins.merge(
      census_df[census_df.median_income <= bot_25_median][["Code", "median_income"]], 
      left_on = "DestCityMarketID", right_on="Code")
    print("Flights originate upper 25 to bottom 25")
    print("Mean of FarePerMile : ", origin_dest.FarePerMile.mean())
    print("Mean of MilesFlown : ", origin_dest.MilesFlown.mean())
    print("Mean of Average Segments:", origin_dest.SeqNum.mean())
    print("FarePerMile by carrier:")
    origin_dest[["FarePerMile", "RPCarrier"]].groupby("RPCarrier").mean()["FarePerMile"].plot(kind="bar")
    plt.show()
    print("FarePerMile Distribution:")
    origin_dest.FarePerMile.hist()
    plt.show()

# This shows the flight statistics of city areas that has 
# with upper25 as both orgin and dest
# and lower as both orgin and dest
def double_low_high(combined, census_df):
    bot_25_median = census_df.median_income.quantile(.25)
    up_25_median = census_df.median_income.quantile(.75)
    origins = combined[["FarePerMile", "RPCarrier", "OriginCityMarketID", 
                                  "DestCityMarketID", "MilesFlown", "SeqNum"]].merge(
      census_df[census_df.median_income <= bot_25_median][["Code", "median_income"]], 
      left_on = "OriginCityMarketID", right_on="Code")
    origin_dest = origins.merge(
      census_df[census_df.median_income <= bot_25_median][["Code", "median_income"]], 
      left_on = "DestCityMarketID", right_on="Code")
    print("Flights originate and destin for bottom 25")
    print("Mean of FarePerMile : ", origin_dest.FarePerMile.mean())
    print("Mean of MilesFlown : ", origin_dest.MilesFlown.mean())
    print("Mean of Average Segments:", origin_dest.SeqNum.mean())
    print("FarePerMile by carrier:")
    origin_dest[["FarePerMile", "RPCarrier"]].groupby("RPCarrier").mean()["FarePerMile"].plot(kind="bar")
    plt.show()
    print("FarePerMile Distribution:")
    origin_dest.FarePerMile.hist()
    plt.show()
    origins = combined[["FarePerMile", "RPCarrier", "OriginCityMarketID", 
                                  "DestCityMarketID", "MilesFlown", "SeqNum"]].merge(
      census_df[census_df.median_income >= up_25_median][["Code", "median_income"]], 
      left_on = "OriginCityMarketID", right_on="Code")
    origin_dest = origins.merge(
      census_df[census_df.median_income >= up_25_median][["Code", "median_income"]], 
      left_on = "DestCityMarketID", right_on="Code")
    print("Flights originate and destin for upper 25")
    print("Mean of FarePerMile : ", origin_dest.FarePerMile.mean())
    print("Mean of MilesFlown : ", origin_dest.MilesFlown.mean())
    print("Mean of Average Segments:", origin_dest.SeqNum.mean())
    print("FarePerMile by carrier:")
    origin_dest[["FarePerMile", "RPCarrier"]].groupby("RPCarrier").mean()["FarePerMile"].plot(kind="bar")
    plt.show()
    print("FarePerMile Distribution:")
    origin_dest.FarePerMile.hist()
    plt.show()


Writing other.py


# T-100 Real Flight and Passenger count data
Useful for getting load factors, and estimating real revenue

In [ ]:
#@title T-100 file
%%writefile t100.py

## T-100
# Please only use the domestic segment data for this module

# dependency
import pandas as pd

# would return only with flights that are schedule for passenger
def import_T100(path):
  segment = pd.read_csv(path)
  print("Successfully imported!")
  # filtering with passenger class, passenger aircraft, and flights that never run
  passenger = segment[(segment.CLASS == "F") & (segment["AIRCRAFT_CONFIG"] == 1) & segment["DEPARTURES_PERFORMED"] != 0]
  return passenger

# A preset of useful columns
def useful_cols(passenger):
  return passenger[["DEPARTURES_PERFORMED", "CARRIER", 'UNIQUE_CARRIER_NAME','SEATS','PASSENGERS'
                    , 'DISTANCE', 'RAMP_TO_RAMP', 'CARRIER_GROUP_NEW', 'ORIGIN_CITY_MARKET_ID', 
                'ORIGIN', 'DEST_CITY_MARKET_ID', 'DEST', 'YEAR', 'QUARTER', 'MONTH', 'DISTANCE_GROUP']]

# Grabbing load factor value series
# note that the orginal dataset passenger and load count a summarized with the same "departure"
# this means that if depart_per = 100, passenger = 10000, per flight passenger avg is 100
def gen_load_factor_series(t100_df):
  return t100_df.PASSENGERS / t100_df.SEATS / t100_df.DEPARTURES_PERFORMED

# t-100 segment should only be merging with coupon as it is per segement
# note that t-100 is annual data and coupon is better be use as a quarter data
# due to its sear size
# note that this would only match with market city pair and give the coupon dataset
# the load factor and departures performed 
# the function would run and grab the load_factor variable and should be 
# able to run with load_factor var in with no error as well
def matching_coupon(t100_df, coupon_df):
  t100_df_load = t100_df.copy()
  t100_df_load["LOAD_FACTOR"] = gen_load_factor_series(t100_df_load)
  t_100_grouped = t_100_df.groupby(['YEAR', 'QUARTER', 'CARRIER','ORIGIN_CITY_MARKET_ID', 
                                    'DEST_CITY_MARKET_ID'])
  # this is the city-pair load factor given on a specific carrier
  t_100_grouped_mean = t_100_grouped.mean()[["LOAD_FACTOR"]]
  # The pass_tran and set_tran means given year, quarter and city-pair the total amount of such available
  t_100_grouped_sum = t_100_grouped.sum()[["PASSENGERS", 'SEATS', 
                                           "DEPARTURES_PERFORMED"]].rename(
                                               columns={"PASSENGERS": "PASSENGERS_TRANS", 'SEATS': "SEATS_TRANS"})
  t_100_stat = t_100_grouped_mean.join(t_100_grouped_sum).reset_index()
  return coupon_df.merge(t_100_stat, how="left", on = ['YEAR', 'QUARTER', 'CARRIER','ORIGIN_CITY_MARKET_ID', 
                                    'DEST_CITY_MARKET_ID'])

Writing t100.py


In [ ]:
#@title Passing functions for All coupon and ticket

%%writefile manager.py

import pandas as pd
import zipfile
from other import gen_ticket_coupon

# This function allow for easy extraction of coupon zip
def load_coupon(path, year, quarter):
  with zipfile.ZipFile(
    "{path_f}/Origin_and_Destination_Survey_DB1BCoupon_{year_f}_{quarter_f}.zip".format(
        path_f = path, year_f = year, quarter_f = quarter),"r") as zip_ref:
    df_path = zip_ref.extract([i for i in zip_ref.namelist() if ".csv" in i][0])
  return df_path # Path of result file

# This function allow for easy access of coupon dataframe
def load_coupon_df(path, year, quarter):
  return pd.read_csv(load_coupon(path, year, quarter))

# This function allow for easy extraction of ticket zip
def load_ticket(path, year, quarter):
  with zipfile.ZipFile(
    "{path_f}/Origin_and_Destination_Survey_DB1BTicket_{year_f}_{quarter_f}.zip".format(
        path_f = path, year_f = year, quarter_f = quarter),"r") as zip_ref:
    df_path = zip_ref.extract([i for i in zip_ref.namelist() if ".csv" in i][0])
  return df_path # Path of result file

# This function allow for easy access of coupon dataframe
def load_ticket_df(path, year, quarter):
  return pd.read_csv(load_ticket(path, year, quarter))

# zip -- boolean --True = is a zip path --False = is a .csv path
# feed func_arg with any variables to succesfully to run your function
def run_result_among_ticket(year_start, quarter_start, year_end, quarter_end, 
                            path, zip, function, *func_arg):
  result_storage = []
  for y in range(year_start, year_end + 1):
    q_loader_s = 1
    q_loader_e = 5
    if y == year_start:
      q_loader_s = quarter_start
    elif y == year_end:
      q_loader_e = quarter_end + 1
    for q in range(q_loader_s, q_loader_e):
      if zip:
        curr_df = load_ticket_df(path, y, q)
      else:
        curr_df = pd.read_csv(
              "{path_f}/Origin_and_Destination_Survey_DB1BTicket_{year_f}_{quarter_f}.{file_type}"
              .format(path_f = path, year_f = y, quarter_f = q, file_type = "zip" if zip else "csv"))
      result_storage.append(function(curr_df, *func_arg))
      del curr_df
  return result_storage

def run_result_among_coupon(year_start, quarter_start, year_end, quarter_end, 
                            path, zip, function, *func_arg):
  result_storage = []
  for y in range(year_start, year_end + 1):
    q_loader_s = 1
    q_loader_e = 5
    if y == year_start:
      q_loader_s = quarter_start
    elif y == year_end:
      q_loader_e = quarter_end + 1
    for q in range(q_loader_s, q_loader_e):
      if zip:
        curr_df = load_coupon_df(path, y, q)
      else:
        curr_df = pd.read_csv(
              "{path_f}/Origin_and_Destination_Survey_DB1BCoupon_{year_f}_{quarter_f}.{file_type}"
              .format(path_f = path, year_f = y, quarter_f = q, file_type = "zip" if zip else "csv"))
      result_storage.append(function(curr_df, *func_arg))
      del curr_df
  return result_storage

def run_result_among_combined(
    year_start, quarter_start, year_end, quarter_end,
    ticket_path, coupon_path, zip, function, *func_arg):
  result_storage = []
  for y in range(year_start, year_end + 1):
    q_loader_s = 1
    q_loader_e = 5
    if y == year_start:
      q_loader_s = quarter_start
    elif y == year_end:
      q_loader_e = quarter_end + 1
    for q in range(q_loader_s, q_loader_e):
      if zip:
        curr_coupon_df = load_coupon_df(coupon_path, y, q)
        curr_ticket_df = load_ticket_df(ticket_path, y, q)
      else:
        curr_coupon_df = pd.read_csv(
              "{path_f}/Origin_and_Destination_Survey_DB1BCoupon_{year_f}_{quarter_f}.{file_type}"
              .format(path_f = coupon_path, year_f = y, quarter_f = q, file_type = "zip" if zip else "csv"))
        curr_ticket_df = pd.read_csv(
              "{path_f}/Origin_and_Destination_Survey_DB1BTicket_{year_f}_{quarter_f}.{file_type}"
              .format(path_f = ticket_path, year_f = y, quarter_f = q, file_type = "zip" if zip else "csv"))
      curr_df = gen_ticket_coupon(curr_ticket_df, curr_coupon_df)
      del curr_ticket_df
      del curr_coupon_df
      result_storage.append(function(curr_df, *func_arg))
      del curr_df
  return result_storage

Writing manager.py


# Grabing race and income bias

In [ ]:
#@title Grabing race and income bias
%%writefile graber.py

import pandas as pd

# getting lookup table for census data
def city_code_reader():
  return pd.read_csv("https://raw.githubusercontent.com/ptse8204/airlinedatabias/main/lookup/city_id_census.csv")